# Data Owner (Client1) – prepare and serve Twin

This notebook loads the private + public single-cell AnnData files, builds a Twin, publishes it, and handles computation requests from Client2.

## Setup
- Optional: install deps if needed.
- Add the project src to `sys.path` and import Beaver.

In [ ]:
!cd shared && ls

In [ ]:
# # Clean start
!cd shared && rm -rf public client1 client2

In [ ]:
# Optional: install if your env doesn't have these yet
# !uv pip install scanpy anndata matplotlib scikit-misc

In [ ]:
# !uv pip install scanpy anndata matplotlib scikit-misc

In [ ]:
# !uv pip install matplotlib

In [ ]:
# !uv pip install numpy

In [ ]:
import sys
from pathlib import Path
import scanpy as sc
from beaver import Twin
import beaver

# Make package importable from repo
sys.path.insert(0, "../../python/src")

bv = beaver.connect("shared", user="client1")

In [ ]:
bv.inbox()

In [ ]:
# bv.inbox()[0].load()

## Load raw/private and mock/public data

In [ ]:
data_dir = Path("../../notebooks/single_cell/data")
private_path = data_dir / "sc_RNAseq_adata_downsampled_to5percent.private.h5ad"
mock_path = data_dir / "sc_RNAseq_adata_downsampled_to5percent.mock.h5ad"
sim_path = data_dir / "adata_simulated.h5ad"

In [ ]:
adata_sim = sc.read(sim_path)

In [ ]:
adata_sim.obs.rename(
    columns={"pct_counts_in_top_50_genes": "pct_counts_mt"},
    inplace=True,
)
adata_sim.obs.rename(columns={"group": "cell_type"}, inplace=True)

In [ ]:
adata_sim.write_h5ad(mock_path)

In [ ]:
adata_private = sc.read(private_path)
adata_mock = sc.read(mock_path)

In [ ]:
# # A way to visualise quality metrics of the matrices
# sc.pl.violin(adata_private, ["n_genes_by_counts", "total_counts", "pct_counts_mt"], size=0, multi_panel=True)

In [ ]:
# # A way to visualise quality metrics of the matrices
# sc.pl.violin(adata_mock, ["n_genes_by_counts", "total_counts", "pct_counts_mt"], size=0, multi_panel=True)

In [ ]:
import anndata as ad
from beaver.runtime import TrustedLoader  # available via bv.type_registry()

In [ ]:
@TrustedLoader.register(ad.AnnData)
def annadata_serialize_file(obj, path):
  # write to the provided path; only the path + loader code is sent
  obj.write_h5ad(path)

@TrustedLoader.register(ad.AnnData)
def annadata_deserialize_file(path):
  return ad.read_h5ad(path)

## Build Twin and share with Client2
- Private side: full dataset
- Public side: simulated/mock dataset
- Publish to registry and send to Client2's inbox

In [ ]:
patient_sc = Twin(
    private=adata_private,
    public=adata_mock,
    owner="client1",
    name="patient_sc",
)

# Publish to public registry and send to Client2
bv.remote_vars["patient_sc"] = patient_sc
# send_result = bv.send(patient_sc, user="client2")
# send_result

In [ ]:
# !uv pip uninstall numpy

In [ ]:
assert False

In [ ]:
# # A way to visualise quality metrics of the matrices
# sc.pl.violin(patient_sc.private, ["n_genes_by_counts", "total_counts", "pct_counts_mt"], size=0, multi_panel=True)

In [ ]:
# !uv pip uninstall matplotlib

In [ ]:
patient_sc.can_send()

## Handle computation requests from Client2
Run this cell whenever new requests arrive to execute them on the private data, review, and approve.

In [ ]:
bv.wait_for_message()

In [ ]:
bv.inbox()

In [ ]:
bv.inbox()[0]

In [ ]:
bv.inbox()[0].load()

In [ ]:
request_make_violin_for_result

In [ ]:
x = request_make_violin_for_result.run_mock()

In [ ]:
x.public_figures[0]

In [ ]:
result = request_make_violin_for_result.run_both()

In [ ]:
result

In [ ]:
result.public_stdout

In [ ]:
result.public_figures[0]

In [ ]:
print(result.private_stdout)
print(result.private_stderr)

In [ ]:
result.private_figures[0]

In [ ]:
result.data.public

In [ ]:
result.data.private

In [ ]:
result.approve()

In [ ]:
bv.wait_for_message()

In [ ]:
bv.inbox()

In [ ]:
bv.inbox()[1].load()

In [ ]:
embedding_result = request_show_embedding_for_embedding_plot.run_both()

In [ ]:
embedding_result.data.private

In [ ]:
embedding_result.private_figures[0]

In [ ]:
embedding_result.approve()

In [ ]:
bv.wait_for_message()

In [ ]:
bv.inbox()

In [ ]:
bv.inbox()[2]

In [ ]:
bv.inbox()[2].load()

In [ ]:
request_plot_pca_variance_for_pca_variance

In [ ]:
pca_result = request_plot_pca_variance_for_pca_variance.run_both()

In [ ]:
pca_result

In [ ]:
pca_result.private_figures[0]

In [ ]:
pca_result.approve()

In [ ]:
bv.wait_for_message()

In [ ]:
bv.inbox()

In [ ]:
bv.inbox()[3]

In [ ]:
bv.inbox()[3].load()

In [ ]:
request_umap_embedding_for_umap_embeddings

In [ ]:
umap_result = request_umap_embedding_for_umap_embeddings.run_both()

In [ ]:
umap_result

In [ ]:
umap_result.data.private

In [ ]:
umap_result.public_figures[0]

In [ ]:
umap_result.private_figures[0]

In [ ]:
umap_result.approve()